In [4]:
import numpy as np
import pandas as pd
import os
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.model_selection import LeaveOneOut, cross_val_score, GroupKFold, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from tabulate import tabulate
from sklearn.metrics import confusion_matrix
from sklearn.feature_selection import SelectKBest, f_classif
from scipy.fft import fft
import time
from pyentrp import entropy as ent
import warnings
from sklearn.exceptions import ConvergenceWarning
import csv
from sklearn.pipeline import Pipeline

In [5]:
# Function to reduce features using SelectKBest
def select_features(X_train, y_train, X_test, k=10):
    # Initialize and fit SelectKBest
    selector = SelectKBest(score_func=f_classif, k=k)
    selector.fit(X_train, y_train)
    
    # Transform both training and testing data
    X_train_reduced = selector.transform(X_train)
    X_test_reduced = selector.transform(X_test)
    
    return X_train_reduced, X_test_reduced

In [9]:
from sklearn.base import BaseEstimator
from sklearn.model_selection import train_test_split
from pytorch_tabnet.tab_model import TabNetClassifier

class TabNetSklearnWrapper(BaseEstimator):
    def __init__(self, verbose=0, max_epochs=100, optimizer_params=dict(lr=2e-2), patience=10, batch_size=256, virtual_batch_size=128):
        self.verbose = verbose
        self.max_epochs = max_epochs
        self.optimizer_params = optimizer_params
        self.patience = patience
        self.batch_size = batch_size
        self.virtual_batch_size = virtual_batch_size
        self.clf = None

    def fit(self, X, y):
        X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)
        self.clf = TabNetClassifier(verbose=self.verbose, optimizer_params=self.optimizer_params)
        self.clf.fit(
            X_train=X_train, y_train=y_train,
            eval_set=[(X_valid, y_valid)],
            eval_name=['valid'],
            eval_metric=['accuracy'],
            max_epochs=self.max_epochs,
            patience=self.patience,
            batch_size=self.batch_size,
            virtual_batch_size=self.virtual_batch_size
        )
        return self

    def predict(self, X):
        return self.clf.predict(X)

    def score(self, X, y):
        preds = self.predict(X)
        return np.mean(preds == y)



def run_tabnet_grid_search(features, labels, log_file):
    pipe = Pipeline([
        ('select_k_best', SelectKBest(score_func=f_classif)),
        ('tabnet', TabNetSklearnWrapper(verbose=0))
    ])

    params = {
    'select_k_best__k': [25, 27, 30, 32, 35, 37, 40],
    'tabnet__max_epochs': [20, 50, 100],
    'tabnet__optimizer_params': [{'lr': 0.01}, {'lr': 0.05}, {'lr': 0.1}, {'lr': 0.5}],
    'tabnet__batch_size': [128, 256, 512],
    'tabnet__virtual_batch_size': [32, 64, 128]
    }



    grid_search = GridSearchCV(pipe, param_grid=params, cv=3, scoring='accuracy', n_jobs=-1)
    try:
        grid_search.fit(features, labels)
        optimized_params = f"Optimized parameters for TabNet: {grid_search.best_params_}\n"
        optimized_score = f"Optimized cross-validation score: {grid_search.best_score_:.2f}\n"
        print(optimized_params)
        print(optimized_score)
        with open(log_file, 'a') as f:
            f.write(optimized_params)
            f.write(optimized_score)
    except Exception as e:
        with open(log_file, 'a') as f:
            f.write(f"Error processing grid search: {str(e)}\n")

def process_file(file_path, log_file):
    df = pd.read_csv(file_path)
    labels = df['label']
    features = df.drop(columns=['subject_id', 'label'])
    run_tabnet_grid_search(features, labels, log_file)

def main(folder_path):
    log_file = os.path.join(os.getcwd(), 'tabnet.txt')
    if os.path.exists(log_file):
        os.remove(log_file)  # Start fresh for each run
    for file_name in os.listdir(folder_path):
        if file_name.startswith("features_") and file_name.endswith(".csv"):
            file_path = os.path.join(folder_path, file_name)
            print(f"\nProcessing file: {file_name}")
            with open(log_file, 'a') as f:
                f.write(f"Processing file: {file_name}\n")
            process_file(file_path, log_file)

main('/home/ximo/Escritorio/ProyectoTFG/featuresExtended')



Processing file: features_1.csv



Early stopping occurred at epoch 11 with best_epoch = 1 and best_valid_accuracy = 0.26042

Early stopping occurred at epoch 11 with best_epoch = 1 and best_valid_accuracy = 0.22917


/home/ximo/.local/lib/python3.10/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
/home/ximo/.local/lib/python3.10/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Stop training because you reached max_epochs = 20 with best_epoch = 12 and best_valid_accuracy = 0.33333
Stop training because you reached max_epochs = 20 with best_epoch = 10 and best_valid_accuracy = 0.35417


/home/ximo/.local/lib/python3.10/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
/home/ximo/.local/lib/python3.10/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
/home/ximo/.local/lib/python3.10/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


...


Early stopping occurred at epoch 35 with best_epoch = 25 and best_valid_accuracy = 0.38542


/home/ximo/.local/lib/python3.10/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)



Early stopping occurred at epoch 42 with best_epoch = 32 and best_valid_accuracy = 0.45486


/home/ximo/.local/lib/python3.10/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Optimized parameters for TabNet: {'select_k_best__k': 30, 'tabnet__batch_size': 128, 'tabnet__max_epochs': 100, 'tabnet__optimizer_params': {'lr': 0.01}, 'tabnet__virtual_batch_size': 32}

Optimized cross-validation score: 0.37

